# logistic_regression.py

In [ ]:
import time

import mxnet as mx
import mxnet.gluon as g
import numpy as np

# define hyperparameters
batch_size = 32
learning_rate = 1e-3
epochs = 100
step = 300
ctx = mx.gpu()


# define data transform
def data_transform(data, label):
    return data.astype(np.float32) / 255, label.astype(np.float32)


# define dataset and dataloader
train_dataset = g.data.vision.MNIST(transform=data_transform)
test_dataset = g.data.vision.MNIST(train=False, transform=data_transform)

train_loader = g.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
test_loader = g.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# define model
logistic_model = g.nn.Sequential()
with logistic_model.name_scope():
    logistic_model.add(g.nn.Dense(10))

logistic_model.collect_params().initialize(
    mx.init.Xavier(rnd_type='gaussian'), ctx=ctx)

criterion = g.loss.SoftmaxCrossEntropyLoss()
optimizer = g.Trainer(logistic_model.collect_params(), 'sgd',
                      {'learning_rate': learning_rate})

# start train
for e in range(epochs):
    print('*' * 10)
    print('epoch {}'.format(e + 1))
    since = time.time()
    moving_loss = 0.0
    moving_acc = 0.0
    for i, (img, label) in enumerate(train_loader, 1):
        img = img.as_in_context(ctx).reshape((-1, 28 * 28))
        label = label.as_in_context(ctx)
        with mx.autograd.record():
            output = logistic_model(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step(img.shape[0])
        # =========== keep average loss and accuracy ==============
        moving_loss += mx.nd.mean(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        acc = mx.nd.mean(predict == label).asscalar()
        moving_acc += acc
        if i % step == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                i, len(train_loader), moving_loss / step, moving_acc / step))
            moving_loss = 0.0
            moving_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
    total = 0.0
    for img, label in test_loader:
        img = img.as_in_context(ctx).reshape((-1, 28 * 28))
        label = label.as_in_context(ctx)
        output = logistic_model(img)
        loss = criterion(output, label)
        test_loss += mx.nd.sum(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        test_acc += mx.nd.sum(predict == label).asscalar()
        total += img.shape[0]
    print('Test Loss: {:.6f}, Test Acc: {:.6f}'.format(test_loss / total,
                                                       test_acc / total))
    print('Time: {:.1f} s'.format(time.time() - since))

logistic_model.save_params('./logistic.params')

C:\ProgramData\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# neural network.py

In [2]:
__author__ = 'SherlockLiao'

import time

import mxnet as mx
import mxnet.gluon as g
import numpy as np

# define hyperparameters
batch_size = 32
learning_rate = 1e-2
epochs = 100
step = 300
ctx = mx.gpu()


# define data transform
def data_transform(data, label):
    return data.astype(np.float32) / 255, label.astype(np.float32)


# define dataset and dataloader
train_dataset = g.data.vision.MNIST(transform=data_transform)
test_dataset = g.data.vision.MNIST(train=False, transform=data_transform)

train_loader = g.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
test_loader = g.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# define model
net = g.nn.Sequential(prefix='nn_')
with net.name_scope():
    net.add(g.nn.Dense(300, activation='relu'))
    net.add(g.nn.Dense(100, activation='relu'))
    net.add(g.nn.Dense(10))

net.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

criterion = g.loss.SoftmaxCrossEntropyLoss()
optimizer = g.Trainer(net.collect_params(), 'sgd',
                      {'learning_rate': learning_rate})

# start train
for e in range(epochs):
    print('*' * 10)
    print('epoch {}'.format(e + 1))
    since = time.time()
    moving_loss = 0.0
    moving_acc = 0.0
    for i, (img, label) in enumerate(train_loader, 1):
        img = img.as_in_context(ctx).reshape((-1, 28 * 28))
        label = label.as_in_context(ctx)
        with mx.autograd.record():
            output = net(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step(img.shape[0])
        # =========== keep average loss and accuracy ==============
        moving_loss += mx.nd.mean(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        acc = mx.nd.mean(predict == label).asscalar()
        moving_acc += acc
        if i % step == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                i, len(train_loader), moving_loss / step, moving_acc / step))
            moving_loss = 0.0
            moving_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
    total = 0.0
    for img, label in test_loader:
        img = img.as_in_context(ctx).reshape((-1, 28 * 28))
        label = label.as_in_context(ctx)
        output = net(img)
        loss = criterion(output, label)
        test_loss += mx.nd.sum(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        test_acc += mx.nd.sum(predict == label).asscalar()
        total += img.shape[0]
    print('Test Loss: {:.6f}, Test Acc: {:.6f}'.format(test_loss / total,
                                                       test_acc / total))
    print('Time: {:.1f} s'.format(time.time() - since))

net.save_params('./nn.params')

c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision\datasets.py:82: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision\datasets.py:86: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


**********
epoch 1
[300/1875] Loss: 1.353705, Acc: 0.665417
[600/1875] Loss: 0.587442, Acc: 0.857708
[900/1875] Loss: 0.436677, Acc: 0.880313
[1200/1875] Loss: 0.374700, Acc: 0.897292
[1500/1875] Loss: 0.364855, Acc: 0.895208
[1800/1875] Loss: 0.326764, Acc: 0.908438
Test Loss: 0.302821, Test Acc: 0.914300
Time: 14.2 s
**********
epoch 2
[300/1875] Loss: 0.311284, Acc: 0.909896
[600/1875] Loss: 0.292147, Acc: 0.917188
[900/1875] Loss: 0.273953, Acc: 0.922083
[1200/1875] Loss: 0.258142, Acc: 0.926146
[1500/1875] Loss: 0.264857, Acc: 0.925521
[1800/1875] Loss: 0.258325, Acc: 0.928229
Test Loss: 0.230882, Test Acc: 0.933100
Time: 13.8 s
**********
epoch 3
[300/1875] Loss: 0.235316, Acc: 0.931354
[600/1875] Loss: 0.227938, Acc: 0.935625
[900/1875] Loss: 0.229813, Acc: 0.936979
[1200/1875] Loss: 0.230178, Acc: 0.934792
[1500/1875] Loss: 0.215781, Acc: 0.938958
[1800/1875] Loss: 0.205387, Acc: 0.940729
Test Loss: 0.197936, Test Acc: 0.941200
Time: 14.0 s
**********
epoch 4
[300/1875] Loss: 0

[1500/1875] Loss: 0.034638, Acc: 0.990521
[1800/1875] Loss: 0.033774, Acc: 0.990625
Test Loss: 0.073365, Test Acc: 0.977100
Time: 13.0 s
**********
epoch 27
[300/1875] Loss: 0.030726, Acc: 0.992604
[600/1875] Loss: 0.031152, Acc: 0.993229
[900/1875] Loss: 0.030027, Acc: 0.992708
[1200/1875] Loss: 0.032512, Acc: 0.992812
[1500/1875] Loss: 0.029141, Acc: 0.993229
[1800/1875] Loss: 0.036274, Acc: 0.990417
Test Loss: 0.070126, Test Acc: 0.978500
Time: 13.5 s
**********
epoch 28
[300/1875] Loss: 0.031071, Acc: 0.992604
[600/1875] Loss: 0.027999, Acc: 0.993229
[900/1875] Loss: 0.028186, Acc: 0.993854
[1200/1875] Loss: 0.030571, Acc: 0.992812
[1500/1875] Loss: 0.028037, Acc: 0.993646
[1800/1875] Loss: 0.031767, Acc: 0.992708
Test Loss: 0.072816, Test Acc: 0.977700
Time: 13.1 s
**********
epoch 29
[300/1875] Loss: 0.027274, Acc: 0.994375
[600/1875] Loss: 0.024041, Acc: 0.995417
[900/1875] Loss: 0.029127, Acc: 0.993854
[1200/1875] Loss: 0.029290, Acc: 0.993021
[1500/1875] Loss: 0.029042, Acc: 0

[300/1875] Loss: 0.008195, Acc: 0.998854
[600/1875] Loss: 0.007536, Acc: 0.999167
[900/1875] Loss: 0.008470, Acc: 0.999479
[1200/1875] Loss: 0.008520, Acc: 0.999271
[1500/1875] Loss: 0.008902, Acc: 0.999583
[1800/1875] Loss: 0.009281, Acc: 0.998854
Test Loss: 0.066925, Test Acc: 0.980600
Time: 12.9 s
**********
epoch 53
[300/1875] Loss: 0.008476, Acc: 0.999479
[600/1875] Loss: 0.007913, Acc: 0.999375
[900/1875] Loss: 0.007642, Acc: 0.999583
[1200/1875] Loss: 0.007405, Acc: 0.999167
[1500/1875] Loss: 0.008703, Acc: 0.999062
[1800/1875] Loss: 0.008166, Acc: 0.999167
Test Loss: 0.066920, Test Acc: 0.980900
Time: 12.9 s
**********
epoch 54
[300/1875] Loss: 0.007974, Acc: 0.999479
[600/1875] Loss: 0.006387, Acc: 0.999792
[900/1875] Loss: 0.006092, Acc: 1.000000
[1200/1875] Loss: 0.007534, Acc: 0.999479
[1500/1875] Loss: 0.007580, Acc: 0.998958
[1800/1875] Loss: 0.009085, Acc: 0.998646
Test Loss: 0.067772, Test Acc: 0.980400
Time: 12.9 s
**********
epoch 55
[300/1875] Loss: 0.007610, Acc: 0.

[1500/1875] Loss: 0.003669, Acc: 0.999896
[1800/1875] Loss: 0.003589, Acc: 0.999896
Test Loss: 0.071340, Test Acc: 0.981000
Time: 13.8 s
**********
epoch 78
[300/1875] Loss: 0.003000, Acc: 1.000000
[600/1875] Loss: 0.003634, Acc: 0.999896
[900/1875] Loss: 0.003294, Acc: 1.000000
[1200/1875] Loss: 0.003253, Acc: 1.000000
[1500/1875] Loss: 0.003293, Acc: 0.999896
[1800/1875] Loss: 0.003528, Acc: 0.999896
Test Loss: 0.071290, Test Acc: 0.981500
Time: 13.2 s
**********
epoch 79
[300/1875] Loss: 0.002767, Acc: 1.000000
[600/1875] Loss: 0.003069, Acc: 1.000000
[900/1875] Loss: 0.003410, Acc: 0.999792
[1200/1875] Loss: 0.003498, Acc: 0.999896
[1500/1875] Loss: 0.003099, Acc: 1.000000
[1800/1875] Loss: 0.003315, Acc: 1.000000
Test Loss: 0.071545, Test Acc: 0.980700
Time: 12.9 s
**********
epoch 80
[300/1875] Loss: 0.002953, Acc: 0.999896
[600/1875] Loss: 0.003127, Acc: 0.999896
[900/1875] Loss: 0.003238, Acc: 0.999896
[1200/1875] Loss: 0.003201, Acc: 0.999896
[1500/1875] Loss: 0.003081, Acc: 1

# convolution_network.py

In [3]:
__author__ = 'SherlockLiao'

import time

import mxnet as mx
import mxnet.gluon as g
import numpy as np

# define hyperparameters
batch_size = 32
learning_rate = 1e-2
epochs = 100
step = 300
ctx = mx.gpu()


# define data transform
def data_transform(data, label):
    return mx.nd.transpose(data.astype(np.float32) / 255,
                           (2, 0, 1)), label.astype(np.float32)


# define dataset and dataloader
train_dataset = g.data.vision.MNIST(transform=data_transform)
test_dataset = g.data.vision.MNIST(train=False, transform=data_transform)

train_loader = g.data.DataLoader(
    train_dataset, batch_size=batch_size, shuffle=True)
test_loader = g.data.DataLoader(
    test_dataset, batch_size=batch_size, shuffle=False)

# define model
cnn = g.nn.Sequential(prefix='cnn_')
with cnn.name_scope():
    cnn.add(g.nn.Conv2D(6, 3, strides=1, padding=1, activation='relu'))
    cnn.add(g.nn.MaxPool2D(2, 2))
    cnn.add(g.nn.Conv2D(16, 5, strides=1, padding=0, activation='relu'))
    cnn.add(g.nn.MaxPool2D(2, 2))
    cnn.add(g.nn.Flatten())
    cnn.add(g.nn.Dense(120, activation='relu'))
    cnn.add(g.nn.Dense(84, activation='relu'))
    cnn.add(g.nn.Dense(10))

cnn.collect_params().initialize(mx.init.Xavier(), ctx=ctx)

criterion = g.loss.SoftmaxCrossEntropyLoss()
optimizer = g.Trainer(cnn.collect_params(), 'sgd',
                      {'learning_rate': learning_rate})

# start train
for e in range(epochs):
    print('*' * 10)
    print('epoch {}'.format(e + 1))
    since = time.time()
    moving_loss = 0.0
    moving_acc = 0.0
    for i, (img, label) in enumerate(train_loader, 1):
        img = img.as_in_context(ctx)
        label = label.as_in_context(ctx)
        with mx.autograd.record():
            output = cnn(img)
            loss = criterion(output, label)
        loss.backward()
        optimizer.step(img.shape[0])
        # =========== keep average loss and accuracy ==============
        moving_loss += mx.nd.mean(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        acc = mx.nd.mean(predict == label).asscalar()
        moving_acc += acc

        if i % step == 0:
            print('[{}/{}] Loss: {:.6f}, Acc: {:.6f}'.format(
                i, len(train_loader), moving_loss / step, moving_acc / step))
            moving_loss = 0.0
            moving_acc = 0.0
    test_loss = 0.0
    test_acc = 0.0
    total = 0.0
    for img, label in test_loader:
        img = img.as_in_context(ctx)
        label = label.as_in_context(ctx)
        output = cnn(img)
        loss = criterion(output, label)
        test_loss += mx.nd.sum(loss).asscalar()
        predict = mx.nd.argmax(output, axis=1)
        test_acc += mx.nd.sum(predict == label).asscalar()
        total += img.shape[0]
    print('Test Loss: {:.6f}, Test Acc: {:.6f}'.format(test_loss / total,
                                                       test_acc / total))
    print('Time: {:.1f} s'.format(time.time() - since))

cnn.save_params('./cnn.params')

c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision\datasets.py:82: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  label = np.fromstring(fin.read(), dtype=np.uint8).astype(np.int32)
c:\anaconda3\lib\site-packages\mxnet\gluon\data\vision\datasets.py:86: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  data = np.fromstring(fin.read(), dtype=np.uint8)


**********
epoch 1
[300/1875] Loss: 1.425269, Acc: 0.544375
[600/1875] Loss: 0.450449, Acc: 0.856354
[900/1875] Loss: 0.317671, Acc: 0.900729
[1200/1875] Loss: 0.252941, Acc: 0.923854
[1500/1875] Loss: 0.231613, Acc: 0.928958
[1800/1875] Loss: 0.196033, Acc: 0.938021
Test Loss: 0.170941, Test Acc: 0.949000
Time: 21.5 s
**********
epoch 2
[300/1875] Loss: 0.166526, Acc: 0.950000
[600/1875] Loss: 0.151132, Acc: 0.954583
[900/1875] Loss: 0.160145, Acc: 0.950313
[1200/1875] Loss: 0.143822, Acc: 0.957396
[1500/1875] Loss: 0.134757, Acc: 0.958542
[1800/1875] Loss: 0.121429, Acc: 0.962187
Test Loss: 0.101339, Test Acc: 0.967900
Time: 21.3 s
**********
epoch 3
[300/1875] Loss: 0.118136, Acc: 0.961875
[600/1875] Loss: 0.098743, Acc: 0.971458
[900/1875] Loss: 0.109343, Acc: 0.965417
[1200/1875] Loss: 0.104917, Acc: 0.968437
[1500/1875] Loss: 0.091441, Acc: 0.970000
[1800/1875] Loss: 0.087949, Acc: 0.971875
Test Loss: 0.085753, Test Acc: 0.972400
Time: 21.3 s
**********
epoch 4
[300/1875] Loss: 0

[1500/1875] Loss: 0.014386, Acc: 0.995521
[1800/1875] Loss: 0.013828, Acc: 0.996146
Test Loss: 0.043133, Test Acc: 0.987700
Time: 21.3 s
**********
epoch 27
[300/1875] Loss: 0.008639, Acc: 0.997708
[600/1875] Loss: 0.011048, Acc: 0.997396
[900/1875] Loss: 0.012004, Acc: 0.996146
[1200/1875] Loss: 0.011653, Acc: 0.995938
[1500/1875] Loss: 0.010652, Acc: 0.997500
[1800/1875] Loss: 0.009894, Acc: 0.996875
Test Loss: 0.040463, Test Acc: 0.988800
Time: 21.3 s
**********
epoch 28
[300/1875] Loss: 0.008646, Acc: 0.997812
[600/1875] Loss: 0.008178, Acc: 0.997917
[900/1875] Loss: 0.009142, Acc: 0.996771
[1200/1875] Loss: 0.014115, Acc: 0.996250
[1500/1875] Loss: 0.009529, Acc: 0.997188
[1800/1875] Loss: 0.007704, Acc: 0.997292
Test Loss: 0.041103, Test Acc: 0.987600
Time: 21.2 s
**********
epoch 29
[300/1875] Loss: 0.010667, Acc: 0.996875
[600/1875] Loss: 0.006873, Acc: 0.998021
[900/1875] Loss: 0.007283, Acc: 0.998229
[1200/1875] Loss: 0.011567, Acc: 0.996771
[1500/1875] Loss: 0.007925, Acc: 0

[300/1875] Loss: 0.001908, Acc: 0.999375
[600/1875] Loss: 0.002206, Acc: 0.999479
[900/1875] Loss: 0.001326, Acc: 0.999687
[1200/1875] Loss: 0.002017, Acc: 0.999792
[1500/1875] Loss: 0.001498, Acc: 0.999687
[1800/1875] Loss: 0.000967, Acc: 1.000000
Test Loss: 0.047593, Test Acc: 0.989900
Time: 21.2 s
**********
epoch 53
[300/1875] Loss: 0.001111, Acc: 0.999896
[600/1875] Loss: 0.000980, Acc: 1.000000
[900/1875] Loss: 0.000883, Acc: 1.000000
[1200/1875] Loss: 0.000798, Acc: 0.999896
[1500/1875] Loss: 0.001455, Acc: 0.999792
[1800/1875] Loss: 0.001040, Acc: 0.999792
Test Loss: 0.049757, Test Acc: 0.988700
Time: 21.4 s
**********
epoch 54
[300/1875] Loss: 0.000986, Acc: 0.999896
[600/1875] Loss: 0.001064, Acc: 0.999792
[900/1875] Loss: 0.000535, Acc: 1.000000
[1200/1875] Loss: 0.000721, Acc: 1.000000
[1500/1875] Loss: 0.001126, Acc: 0.999792
[1800/1875] Loss: 0.000800, Acc: 1.000000
Test Loss: 0.048962, Test Acc: 0.989100
Time: 21.2 s
**********
epoch 55
[300/1875] Loss: 0.000661, Acc: 1.

[1500/1875] Loss: 0.000219, Acc: 1.000000
[1800/1875] Loss: 0.000321, Acc: 1.000000
Test Loss: 0.054955, Test Acc: 0.989200
Time: 21.2 s
**********
epoch 78
[300/1875] Loss: 0.000199, Acc: 1.000000
[600/1875] Loss: 0.000202, Acc: 1.000000
[900/1875] Loss: 0.000231, Acc: 1.000000
[1200/1875] Loss: 0.000235, Acc: 1.000000
[1500/1875] Loss: 0.000251, Acc: 1.000000
[1800/1875] Loss: 0.000246, Acc: 1.000000
Test Loss: 0.055760, Test Acc: 0.989200
Time: 21.2 s
**********
epoch 79
[300/1875] Loss: 0.000224, Acc: 1.000000
[600/1875] Loss: 0.000174, Acc: 1.000000
[900/1875] Loss: 0.000211, Acc: 1.000000
[1200/1875] Loss: 0.000268, Acc: 1.000000
[1500/1875] Loss: 0.000256, Acc: 1.000000
[1800/1875] Loss: 0.000281, Acc: 1.000000
Test Loss: 0.055213, Test Acc: 0.989000
Time: 21.1 s
**********
epoch 80
[300/1875] Loss: 0.000200, Acc: 1.000000
[600/1875] Loss: 0.000209, Acc: 1.000000
[900/1875] Loss: 0.000194, Acc: 1.000000
[1200/1875] Loss: 0.000246, Acc: 1.000000
[1500/1875] Loss: 0.000251, Acc: 1